In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt

In [ ]:
ds = load_dataset("htdung167/ViHSD")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df_train = pd.DataFrame(ds['train'])
df_valid = pd.DataFrame(ds['validation'])
df_test = pd.DataFrame(ds['test'])

# Convert to binary_label

In [ ]:
binary_mapping = {0: 0, 1: 1, 2: 1}

df_train['binary_label'] = df_train['label_id'].map(binary_mapping)
df_valid['binary_label'] = df_valid['label_id'].map(binary_mapping)
df_test['binary_label'] = df_test['label_id'].map(binary_mapping)

In [ ]:
df_train.drop(columns=['label_id'], inplace=True)
df_valid.drop(columns=['label_id'], inplace=True)
df_test.drop(columns=['label_id'], inplace=True)

In [ ]:
df_train.dtypes

,0
free_text,object
binary_label,int64


In [ ]:
df_train.head()

,free_text,binary_label
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,1
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,1
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0


In [ ]:
file_path = "/content/drive/My Drive/Colab Notebooks/Dataset process 1 (RA)/toxic_data_process2.csv"
df_all_toxic = pd.read_csv(file_path)
df_all_toxic.head()

,toxic_text,paraphased_text
0,bấp bênh vl thế,NaN
1,chắc cũng biết ko tồn tại đc bao lâu ...,NaN
2,thấy chán ad page này kiến thức thì n...,NaN
3,đcm 😒 sau có con cho hút cỏ chữa bệnh c...,NaN
4,má nứng quá aiu ơi,NaN


In [ ]:
df_all_toxic.drop(columns=['paraphased_text'], inplace=True)

In [ ]:
df_all_toxic

,toxic_text
0,bấp bênh vl thế
1,chắc cũng biết ko tồn tại đc bao lâu ...
2,thấy chán ad page này kiến thức thì n...
3,đcm 😒 sau có con cho hút cỏ chữa bệnh c...
4,má nứng quá aiu ơi
...,...
10062,hải yến đéo care cmt ăn lốn màu à con điên
10063,cám ơn đảng cám ơn nhà nc
10064,khẩu nghiệp quá con cháu nghe xấu hổ chết
10065,tôi không dùng bảo hiểm bởi vì bác sĩ coi thườ...


In [ ]:
df_all_toxic['binary_label'] = 1
df_all_toxic.dtypes

,0
toxic_text,object
binary_label,int64


In [ ]:
print("Train:\n", df_train['binary_label'].value_counts(normalize=True) * 100)
print("Valid:\n", df_valid['binary_label'].value_counts(normalize=True) * 100)
print("Test:\n", df_test['binary_label'].value_counts(normalize=True) * 100)


Train:
 binary_label
0    82.692947
1    17.307053
Name: proportion, dtype: float64
Valid:
 binary_label
0    81.961078
1    18.038922
Name: proportion, dtype: float64
Test:
 binary_label
0    83.053892
1    16.946108
Name: proportion, dtype: float64


In [ ]:
deficit_train = len(df_train[df_train['binary_label'] == 0]) - len(df_train[df_train['binary_label'] == 1])
print(deficit_train)
deficit_valid = len(df_valid[df_valid['binary_label'] == 0]) - len(df_valid[df_valid['binary_label'] == 1])
print(deficit_valid)
deficit_test  = len(df_test[df_test['binary_label'] == 0])  - len(df_test[df_test['binary_label'] == 1])
print(deficit_test)

15724
1708
4416


In [ ]:
# Tổng số mẫu cần thêm vào các tập
total_deficit = deficit_train + deficit_valid + deficit_test

# Nếu thiếu dữ liệu, chia theo tỉ lệ
if total_deficit > len(df_all_toxic):
    ratio_train = deficit_train / total_deficit
    ratio_valid = deficit_valid / total_deficit
    ratio_test  = deficit_test / total_deficit

    # Số mẫu có thể lấy từ df_all_toxic
    deficit_train = int(ratio_train * len(df_all_toxic))
    deficit_valid = int(ratio_valid * len(df_all_toxic))
    deficit_test  = len(df_all_toxic) - deficit_train - deficit_valid  # Đảm bảo tổng đúng

# Chia dữ liệu từ df_all_toxic
df_toxic_train = df_all_toxic.sample(n=deficit_train, random_state=42)
df_all_toxic = df_all_toxic.drop(df_toxic_train.index)

df_toxic_valid = df_all_toxic.sample(n=deficit_valid, random_state=42)
df_all_toxic = df_all_toxic.drop(df_toxic_valid.index)

df_toxic_test = df_all_toxic.sample(n=deficit_test, random_state=42)


In [ ]:
print(df_toxic_train.shape, df_toxic_valid.shape, df_toxic_test.shape)


(7245, 2) (787, 2) (2035, 2)


# Concation Dataset

In [ ]:
df_train = df_train.rename(columns={'free_text': 'text', 'binary_label': 'label'})
df_valid = df_valid.rename(columns={'free_text': 'text', 'binary_label': 'label'})
df_test = df_test.rename(columns={'free_text': 'text', 'binary_label': 'label'})

df_toxic_train = df_toxic_train.rename(columns={'toxic_text': 'text', 'binary_label': 'label'})
df_toxic_valid = df_toxic_valid.rename(columns={'toxic_text': 'text', 'binary_label': 'label'})
df_toxic_test = df_toxic_test.rename(columns={'toxic_text': 'text', 'binary_label': 'label'})


In [ ]:
df_train = pd.concat([df_train, df_toxic_train], ignore_index=True)
df_valid = pd.concat([df_valid, df_toxic_valid], ignore_index=True)
df_test  = pd.concat([df_test, df_toxic_test], ignore_index=True)

In [ ]:
print(f"Train set: {df_train['label'].value_counts()}")
print(f"Valid set: {df_valid['label'].value_counts()}")
print(f"Test set: {df_test['label'].value_counts()}")


Train set: label
0    19886
1    18652
Name: count, dtype: int64
Valid set: label
0    2190
1    2056
Name: count, dtype: int64
Test set: label
0    5548
1    5202
Name: count, dtype: int64


In [ ]:
df_train.to_csv("df_train.csv", index=False)
df_valid.to_csv("df_valid.csv", index=False)
df_test.to_csv("df_test.csv", index=False)

# Tải xuống về máy
files.download("df_train.csv")
files.download("df_valid.csv")
files.download("df_test.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>